# Анализ уровня потребительской лояльности

Описание проекта

Заказчик исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. 
Компании требуется  определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.
Компания провела опрос, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».


Наша задача подготовить дашборд с итогами опроса. Данные выгружены  в SQLite.


Резултаты опроса разделены на категории:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).



### Загрузим данные.

#### Импортируем библиотеки.

In [15]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

#### Подключимся к базе данных и выгрузим необходимые данные.

In [6]:
# путь к БД 
path_to_db_local = 'telecomm_csi.db'

# итоговый путь к БД
path_to_db = None

# если путь на  компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local

elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# если итоговый путь не пустой
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)  

In [17]:
query = """
WITH
u AS (SELECT *,
          CASE 
            WHEN lt_day<=365 THEN 'new'
            WHEN lt_day>365 THEN 'old'
            END as is_new,
            
          CASE
            WHEN nps_score <= 6 THEN 'detractors'
            WHEN nps_score >= 7 AND nps_score <= 8 THEN 'passives'
            WHEN nps_score >=9 THEN 'promoters'
            END as nps_group,
          CASE
            WHEN gender_segment=1 THEN 'female'
            WHEN gender_segment=0 THEN 'male'
            END as gender_segment_gender
      FROM user)
    
SELECT user_id,
       lt_day,
       is_new,
       age,
       gender_segment_gender AS gender_segment,
       os_name,
       cpe_type_name,
       country,
       city,
       SUBSTRING(a_g.title, 4) AS age_segment,
       SUBSTRING( t_g.title, 4) AS traffic_segment,
       SUBSTRING( l_g.title, 4) AS lifetime_segment,
       nps_score,
       nps_group
       
FROM u JOIN location AS l ON u.location_id = l.location_id
       JOIN age_segment AS a_g ON u.age_gr_id = a_g.age_gr_id
       JOIN traffic_segment AS t_g ON u.tr_gr_id = t_g.tr_gr_id
       JOIN lifetime_segment AS l_g ON u.lt_gr_id = l_g.lt_gr_id;
"""

In [18]:
df = pd.read_sql(query, engine) 

#### Сохраним данные в формате csv для дальнейшей работы в Tableau Public.

In [19]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

###  Ссылка на дашборд на сайте Tableau Public:

Дашборд: <https://public.tableau.com/app/profile/anna6845/viz/NPSproject/NPS_1?publish=yes>

### Ссылку на pdf-файл с презентацией:

Презентация: <https://disk.yandex.ru/i/ayPkXKNt1Lvwww>